In [2]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
## Before we continue, we gather code from 8.3.5 (2021)
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def read_in():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ## We refer to 8.2.2 (2021) code that imports Kaggle data, assigning variables as needed.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)
    
    # Open and read the Wikipedia data JSON file.
    ## We refer to 8.2.1 (2021) to import Wikipedia data from the JSON file.
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    # 3. Write a list comprehension to filter out TV shows.
    ## We'll use code from 8.3.3 (2021) that filtered out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    ## We will refactor code from 8.3.5 (2021) for getting cleaned movies.
    clean_movies_list = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    ## We will name our new DataFrame wiki_movies_clean.
    wiki_movies_clean = pd.DataFrame(clean_movies_list)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    ## Per 8.3.7 (2021), we will extract IMDb ID's with the help of a regular expression.
    try:
        wiki_movies_clean['imdb_id'] = wiki_movies_clean['imdb_link'].str.extract(r'(tt\d{7})')
        print(len(wiki_movies_clean))
        wiki_movies_clean.drop_duplicates(subset='imdb_id', inplace=True)
        print(len(wiki_movies_clean))
        wiki_movies_clean.head()
    
    ## Like in 6.2.6 (2021), we print an error statement and have the code pass on to the next row of data.
    except:
        print("IMDb ID not found.")
        pass

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    ## Like in 8.3.7 (2021), we will create a list comprehension to pass through our cleaned movies dataframe
    ## to weed out any null values.
    wiki_columns_to_keep = [column for column in wiki_movies_clean.columns if wiki_movies_clean[column].isnull().sum() < len(wiki_movies_clean) * 0.9]
    wiki_movies_clean = wiki_movies_clean[wiki_columns_to_keep]
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    ## Like in 8.3.8 (2021), we will create a box_office variable that holds non-null box office values.
    ## This time, though, we will look into the clean movies dataframe we made.
    clean_box_office = wiki_movies_clean['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    ## We can refactor code used in 8.3.8 (2021) for our particular box_office variable.
    clean_box_office[clean_box_office.map(lambda x: type(x) != str)]
    clean_box_office = clean_box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    ## For the parsing of box office data, we can refactor our code from 8.3.10 (2021).
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    clean_form_one = r'\$\d+\.?\d*\s*[mb]illion'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    clean_form_two = r'\$\d{1,3}(?:,\d{3})+'
    
    # 12. Add the parse_dollars function.
    ## We define the parse_dollars the same way we did back in 8.3.10 (2021)
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan
    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
        # convert to float and multiply by a million
            value = float(s) * 10**6
        # return value
            return value
    # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
        # convert to float and multiply by a billion
            value = float(s) * 10**9
        # return value
            return value
    # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        # remove dollar sign and commas
            s = re.sub('\$|,','', s)
        # convert to float
            value = float(s)
        # return value
            return value
    # otherwise, return NaN
        else:
            return np.nan
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    ## We will refactor code from 8.3.10 (2021) to output wiki_movies_clean with even cleaner box office data.
    wiki_movies_clean['clean_box_office'] = clean_box_office.str.extract(f'({clean_form_one}|{clean_form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_clean.drop('Box office', axis=1, inplace=True)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    ## First, as in 8.3.11 (2021), we clean up the budget column.
    clean_budget = wiki_movies_clean['Budget'].dropna()
    ## Second, we use a lambda function to turn the budget data into strings for our future regexes.
    clean_budget = clean_budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    ## Next, we tidy up the budget figures.
    clean_budget = clean_budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    ## Then, we remove any citation references that make it hard to read the data.
    clean_budget = clean_budget.str.replace(r'\[\d+\]\s*', '')
    ## Finally, we parse the clean budget data from wiki_movies_clean.
    wiki_movies_clean['clean_budget'] = clean_budget.str.extract(f'({clean_form_one}|{clean_form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_clean.drop('Budget', axis=1, inplace=True)
    
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    ## Continuing on 8.3.11 (2021), we refactor the release date code for our wiki_movies_clean dataframe.
    release_date = wiki_movies_clean['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    ## We will be parsing four forms of release dates as defined below.
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    ## With to_datetime, we can extract the date forms without issue.
    wiki_movies_clean['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    ## Rounding out 8.3.11 (2021), we create a variable where we'll store running time data.
    running_time = wiki_movies_clean['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    ## We also create a variable to match the time patterns, per 8.3.11 (2021).
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    ## Now we extract the running times from wiki_movies_clean
    wiki_movies_clean['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
   
    # 2. Clean the Kaggle metadata.
    movies_df[['title_wiki','title_kaggle']]
    ## In 8.4.1 (2021), we found alot of nonvideo movies.
    ## So we'll set all those movies as videos.
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    ## We'll also adjust the column values as below.
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    pd.to_datetime(ratings['timestamp'], unit='s')
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    pd.options.display.float_format = '{:20,.2f}'.format


    # 3. Merged the two DataFrames into the movies DataFrame.
    ## Per 8.4.1 (2021), we perform an inner join of Wikipedia and Kaggle data.
    ## We only want movie data found in both data sets, so an inner join is enough.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
    movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]
    
    
    # 4. Drop unnecessary columns from the merged DataFrame.
    ## As in 8.4.1 (2021) and 8.4.2 (2021), we find that we can keep the kaggle titles as the movie titles,
    ## so we can drop the Wikipedia titles.
    movies_df = movies_df['title_wiki'].drop
    movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)
    
    # 5. Add in the function to fill in the missing Kaggle data.
    

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.


    # 7. Filter the movies DataFrame for specific columns.


    # 8. Rename the columns in the movies DataFrame.


    # 9. Transform and merge the ratings DataFrame.
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '../Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()